In [1]:
!pip install tensorflow
!pip install keras
!pip install pandas
!pip install hazm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import numpy as np
from hazm import *
import re
import tensorflow as tf
from keras.models import Sequential, save_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [2]:
dataset = pd.read_csv("hamshahri.csv")
print(dataset.head())

      DID      Date    Cat                                               Text
0     1S1  75\04\02  adabh  جاودانگي در زندگي گروهي از طريق هنر \nنگاهي به...
1  DID2S1  75\04\02  adabh  رويدادهاي هنري جهان \nنمايشگاه هنر در خدمت ديك...
2  DID3S1  75\04\02  adabh  برديوار نگارخانه ها \nگالري گلستان: \nنمايشگاه...
3  DID4S1  75\04\02  ejtem  بازي را جدي بگيريم \nمطالعه اي مقدماتي پيرامون...
4  DID5S1  75\04\02  elmfa  تخته سياه و غباري كه سترده نمي شود... \nاشاره;...


In [6]:
data_len = dataset.shape[0]
print('Dataset has {} rows.'.format(data_len))

Dataset has 165233 rows.


In [5]:
cats = dataset["Cat"]
texts = dataset["Text"]
unique_cats = np.unique(cats)
cat_texts_count = dict.fromkeys(unique_cats, 0)

for cat in cats:
  cat_texts_count[cat] += 1

for cat, cnt in cat_texts_count.items():
  print('Category "{}" contains {} posts.'.format(cat, cnt))

Category ". راسته: . شب كلاه شكلان (Semaeostomae)" contains 1 posts.
Category "Adabh" contains 13 posts.
Category "Aeqts" contains 10 posts.
Category "Akhar" contains 10 posts.
Category "Busiw" contains 9 posts.
Category "Cinew" contains 3 posts.
Category "Cultw" contains 651 posts.
Category "Econo" contains 6 posts.
Category "Eqtes" contains 14 posts.
Category "Globa" contains 4 posts.
Category "Gozar" contains 3 posts.
Category "Kharj" contains 22 posts.
Category "Lifew" contains 3 posts.
Category "Musical" contains 3 posts.
Category "Newsp" contains 9 posts.
Category "Nnaft" contains 6 posts.
Category "Polig" contains 2 posts.
Category "Santj" contains 4 posts.
Category "Sciew" contains 6 posts.
Category "Shari" contains 8 posts.
Category "Siasi" contains 18 posts.
Category "Sporw" contains 50 posts.
Category "Theatre" contains 3 posts.
Category "Thought" contains 2 posts.
Category "abksh" contains 495 posts.
Category "adabh" contains 5171 posts.
Category "adarman" contains 31 posts

In [6]:
with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stopwords = fa_stop_words.read().splitlines()

In [7]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()

def preprocessing(text):
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = normalizer.normalize(text)
    text = ' '.join(word for word in text.split() if word not in stopwords)
    return text

## PreProcess

In [11]:
num_classes = len(unique_cats)

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dataset["Cat"].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(dataset["Cat"].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 155 unique tokens.
Shape of data tensor: (165233, 250)


In [13]:
rows_of_train = 250000
rows_of_test = 75000

Y = pd.get_dummies(dataset['Cat']).values
print('Shape of label tensor:', Y.shape)

X_train = X[0:rows_of_train]
X_test = X[rows_of_train: rows_of_train + rows_of_test]

Y_train = Y[0:rows_of_train]
Y_test = Y[rows_of_train:rows_of_train + rows_of_test]

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Shape of label tensor: (165233, 113)
(165233, 250) (165233, 113)
(0, 250) (0, 113)


## Classification

In [14]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(113, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

epochs = 3
batch_size = 64
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

history = model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=epochs,batch_size=batch_size)

2022-07-14 17:17:06.222668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-14 17:17:06.222699: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-14 17:17:06.222723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (milad): /proc/driver/nvidia/version does not exist
2022-07-14 17:17:06.223856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-14 17:17:06.252822: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 20000000 exceeds 10% 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 113)               11413     
                                                                 
Total params: 5,091,813
Trainable params: 5,091,813
Non-trainable params: 0
_________________________________________________________________


2022-07-14 17:17:06.668610: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 165233000 exceeds 10% of free system memory.
2022-07-14 17:17:06.838252: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18671329 exceeds 10% of free system memory.


Epoch 1/3
2582/2582 [==============================] - 799s 309ms/step - loss: 0.3403 - accuracy: 0.9374
Epoch 2/3
2582/2582 [==============================] - 754s 292ms/step - loss: 0.0192 - accuracy: 0.9961
Epoch 3/3
2582/2582 [==============================] - 909s 352ms/step - loss: 0.0140 - accuracy: 0.9964
